In [1]:
import h5py
import numpy as np
filename = "D:\Shetty_data\data_labels\data_labels.h5"

In [2]:
with h5py.File(filename, "r") as f:

    # Get all the labels
    all_sat_LLAHTR = list(f["all_sat_LLAHTR"])
    all_uav_LLAHTR = list(f["all_uav_LLAHTR"])
    all_uav_xyzHTR = list(f["all_uav_xyzHTR"])
    match_array_40 = np.array(list(f["match_array_40"]))
    sat300_image_paths = list(f["sat300_image_paths"])
    uav_image_paths = list(f["uav_image_paths"])


In [3]:
import cv2
from PIL import Image
from torchvision import datasets, models, transforms

In [4]:
path = "D:/Shetty_data/train/"

uav_path = path+uav_image_paths[0].decode("utf-8")
sat_path = path+sat300_image_paths[0].decode("utf-8")

print(np.where(match_array_40[0] == True)[0])

uav_img = Image.open(uav_path).convert("RGB")
sat_img = Image.open(sat_path).convert("RGB")

to_tensor = transforms.ToTensor()

uav_tensor = to_tensor(uav_img)
sat_tensor = to_tensor(sat_img)




[ 0 10 15]


In [5]:
index = 22

ground_height = all_sat_LLAHTR[index][2]-300
print(all_uav_LLAHTR[index][2]-ground_height)

139.67791295928356


In [6]:

import random
import math

uav_image_data = []
matching_sat_data = []
non_matching_sat_data = []
uav_grid = []
uav_zHT = []

for i in range(len(uav_image_paths)):

    matches = np.where(match_array_40[i] == True)[0]

    no_match = random.randint(0,len(uav_image_paths))

    while no_match in matches:
        no_match = random.randint(0,len(uav_image_paths))

    uav_path = path+uav_image_paths[i].decode("utf-8")
    sat_path = path+sat300_image_paths[i].decode("utf-8")
    non_matching_sat = path+sat300_image_paths[no_match].decode("utf-8")

    uav_img = Image.open(uav_path).convert("RGB")
    sat_img = Image.open(sat_path).convert("RGB")
    non_matching_sat_img = Image.open(non_matching_sat).convert("RGB")

    to_tensor = transforms.ToTensor()

    uav_tensor = to_tensor(uav_img)
    sat_tensor = to_tensor(sat_img)
    non_matching_sat_tensor = to_tensor(non_matching_sat_img)

    # sat images were captured 300m above ground
    ground_height = all_sat_LLAHTR[i][2]-300
    
    #calculate relative UAV height
    all_uav_xyzHTR[i][2] = all_uav_LLAHTR[i][2] - ground_height

    x = all_uav_xyzHTR[i][0]
    y = all_uav_xyzHTR[i][1]

    if x >= 200:
        x = 199
    elif x <= -200:
        x = -199

    if y >= 200:
        y = 199
    elif y <= -200:
        y = -199


    x_grid = math.floor(4 + (x / 50)) if x > 0 else math.ceil(3 + (x / 50))
    y_grid = math.floor(4 - (y / 50)) if y > 0 else math.floor(4 - (y / 50))

    pos = 8*y_grid + x_grid

    # label = [0]*64

    # label[pos] = 1

    # uav_image_data.append(uav_tensor)
    # matching_sat_data.append(sat_tensor)
    # non_matching_sat_data.append(non_matching_sat_tensor)
    uav_grid.append(pos)
    uav_zHT.append(list(all_uav_xyzHTR[i][2:]))

    if i%1000 == 0:
        print(i)



    



0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000


In [7]:
import pickle

# with open('D:/Shetty_data/train/uav_image_data.pickle', 'wb') as handle:
#     pickle.dump(uav_image_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('D:/Shetty_data/train/matching_sat_data.pickle', 'wb') as handle:
#     pickle.dump(matching_sat_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('D:/Shetty_data/train/non_matching_sat_data.pickle', 'wb') as handle:
#     pickle.dump(non_matching_sat_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('D:/Shetty_data/train/uav_grid_labels.pickle', 'wb') as handle:
    pickle.dump(uav_grid, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('D:/Shetty_data/train/uav_zht_labels.pickle', 'wb') as handle:
    pickle.dump(uav_zHT, handle, protocol=pickle.HIGHEST_PROTOCOL)
